# 1. Imports

In [ ]:
import pandas as pd
import json
import os
import copy
import time

import seaborn as sns
import matplotlib.pyplot as plt

# 2. Loading data

## 2.1 Identify id:s for Liverpool games

In [ ]:
path = 'C:\\Users\\johnl\\Documents\\GitHub\\opendata\\data\\matches.json'

with open(path) as f:
    data = json.load(f)

print(json.dumps(data, indent=4, sort_keys=True))

# Manually found that the Liverpool games has id's 4039 and 2440

In [ ]:
path = 'C:\\Users\\johnl\\Documents\\GitHub\\opendata\\data\\matches'

id1 = '\\4039'
id2 = '\\2440'

## 2.2 Create player dict

In [ ]:
with open(path + id1 + '\\match_data.json') as f:
    data = json.load(f)
    
lfc_dict = {}
lfc_list = []
if data['id'] == 4039:
    lfc_id = data['away_team']['id']
    for player in data['players']:
        if player['team_id'] == lfc_id:
            obj_id = player['trackable_object']
            first_name = player['first_name']
            last_name = player['last_name']
            lfc_dict[obj_id] = first_name + ' ' + last_name
            lfc_list.append(obj_id)

lfc_dict
lfc_list

name_dict = {}
for player in data['players']:
    obj_id = player['trackable_object']
    first_name = player['first_name']
    last_name = player['last_name']
    name_dict[obj_id] = first_name + ' ' + last_name
name_dict

## 2.3 Load and explore tracking data 

In [ ]:
with open(path + id1 + '\\structured_data.json') as f:
    data = json.load(f)

for frame in data[1000]:
    print(frame)

In [ ]:
players = [i for i in name_dict.keys()]
# cols = ['frame'] + players

In [ ]:
data[1000]['data'][0]

In [ ]:
df = pd.DataFrame(columns=players)
print(f'Number of frames: {len(data)}')
print(f'Number of columns: {len(players)}')

In [ ]:
# s = pd.Series([None for i in range(len(players))], index=players)
# for i in range(len(data)):
#     df.append(s, ignore_index = True)
df

In [ ]:
new_frame = {}
for i in players:
    new_frame[i] = None

t0 = time.time()

li =  []
for frame in data:
    frame_nr = frame['frame']
    curr_frame = copy.deepcopy(new_frame)
    for obj in frame['data']:
        x = obj['x']
        y = obj['y']
        try:
            obj_id = obj['trackable_object']
            curr_frame[obj_id] = (x, y)
        except:
            None
    row = pd.Series(curr_frame, name = frame_nr)
    li.append(curr_frame)

df = pd.DataFrame(li)
#     if frame_nr % 1000 == 0 :
#         t = time.time() - t0
#         print(f'{frame_nr} frames took {t} seconds to process')
print(time.time()-t0)

# df.to_pickle(id1 +'_df')
# df = pd.read_pickle(file_name)

In [ ]:
col_list = []
p_list = []
for col in df.columns:
    player_df = df[col]
    p = len([i for i in player_df if i!= None])/len(player_df)
    # print(col, p)
    col_list.append(str(col))
    p_list.append(p)

plt.bar(x=col_list, height=p_list)

In [ ]:
lfc_df = df[lfc_list]

for col in lfc_df.columns:
    player_df = df[col]
    p = len([i for i in player_df if i!= None])/len(player_df)
    if p == 0.0:
        lfc_df = lfc_df.drop(columns=col)

lfc_df

In [ ]:
col_list = []
p_list = []
for col in lfc_df.columns:
    player_df = lfc_df[col]
    p = len([i for i in player_df if i!= None])/len(player_df)
    # print(col, p)
    col_list.append(str(col))
    p_list.append(p)

plt.bar(x=col_list, height=p_list)